In [2]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


In [127]:

# wybór cech poprzez dodawanie
# funkcja sprawdza czy działanie modelu poprawia się gdy pewna cecha zostanie dodana do zbioru używanch przy klasyfikacji cech
# wybrana zostaje ta cecha która najbardziej poprawiła działanie
# poprawienie trafności musi być wyższe od ustalonego progu - kryterium stopu 

def add_feature(acc, stop_threshold_add, model, all_features, selected_features,  X, y):
        best_feature = None
        best_accuracy = acc
        prev_accuracy = acc

        #wybor zestawu cech do sprawdzenia(wszystkie jeszcze nie używane)
        features_left = all_features - set(selected_features)

        for feature in features_left:
            current_features = selected_features + [feature]
            X_subset = X[:, current_features]

            #podzielenie danych na zbiór uczący i testowy, stworzenie modelu
            X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3, random_state=61)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)

            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_feature = feature
            
            

        #obliczenie poprawy trafności - kryterium stopu
        accuracy_diff =   best_accuracy - prev_accuracy
        #print(f"accuracy_prev: {prev_accuracy}")
        #print(f"accuracy best: {best_accuracy}")
        #print(f"accuracy_diff: {accuracy_diff}")
        
        
        if accuracy_diff > stop_threshold_add:
            selected_features.append(best_feature)
            print(f"Dodano ceche {best_feature}, osiągnięto trafność {best_accuracy}")
            return best_accuracy
        else:
            print(f" Sprawdzono ceche {best_feature}, nie spełniono kryterium.")
            return prev_accuracy

In [135]:
# wybór cech poprzez odejmowanie
# funkcja sprawdza czy działanie modelu poprawia się gdy pewna cecha zostanie usunięta zbioru używanch przy klasyfikacji cech
# wybrana zostaje ta cecha której usunięcie najbardziej poprawiło działanie
# poprawienie trafności musi być wyższe od ustalonego progu - kryterium stopu 

def remove_feature(acc, stop_threshold_remove, model,  selected_features,  X, y):
        worst_feature = None
        best_accuracy = acc
        prev_accuracy = acc
        
        for feature in selected_features:
            current_features = set(selected_features) - {feature}
            X_subset = X[:, list(current_features)]

            #podzielenie danych na zbiór uczący i testowy, stworzenie modelu
            X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.3, random_state=61)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            #obliczenie  trafności
            accuracy = accuracy_score(y_test, y_pred)
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                worst_feature = feature

        #obliczenie poprawy trafności - kryterium stopu
        accuracy_diff =   best_accuracy - prev_accuracy
        
        if accuracy_diff > stop_threshold_remove:
            selected_features.remove(worst_feature)
            print(f"Usunieto ceche {worst_feature}, osiągnieto trafność {best_accuracy}")
            return best_accuracy
        else:
            print(f" Sprawdzono ceche {worst_feature}, nie spełniono kryterium.")
            return prev_accuracy
        




In [148]:
#dodaj czy dodaje usuwa czy oba
def run_selection(mode,accuracy,stop_threshold_add,stop_threshold_remove,model,all_features,  X, y):
        selected_features = []
        current_accuracy = accuracy

        if mode == "add" or mode == "both":
                print("Dodawanie cech!")
                while True:
                        new_accuracy = add_feature(current_accuracy, stop_threshold_add, model, all_features,selected_features,  X, y)

                        #jeśli trafność nie zwieksza się o wartość powyżej progu, cechy przestają być dodawane
                        if(new_accuracy>current_accuracy):
                                current_accuracy = new_accuracy
                        else: break

        if mode == "remove" or mode == "both":
                print("Usuwanie cech!")
                if mode == "remove": selected_features = all_features
                while True:
                        if (len(selected_features) < 2): break

                        new_accuracy = remove_feature(current_accuracy, stop_threshold_remove, model, selected_features,  X, y)

                        #jeśli trafność nie zwieksza się o wartość powyżej progu, cechy przestają być usuwane
                        if(new_accuracy>current_accuracy):
                                current_accuracy = new_accuracy
                        else: break

        
        

        return selected_features

In [1]:
#ładowanie danych z wybranego zbioru danych 
#dataset = load_iris()
dataset = load_wine()

X = dataset.data
y = dataset.target



#tworzenie modelu do testowania
model = GaussianNB()

#parametry
threshold_add=0.01
threshold_remove=0.05
starting_accuracy = 0

#zestawy cech
all_features = set(range(X.shape[1]))


mode = 'both' #both/add/remove

selected_features = run_selection(mode,starting_accuracy, threshold_add, threshold_remove, model, all_features, X, y)

print("Wybrane cechy:", selected_features)

NameError: name 'load_wine' is not defined